In [15]:
import numpy as np
import matplotlib.pyplot as plt

## Beachte Konvention: Zeit in Stunden und Winkel in Grad

### Observed Zenith angle

In [16]:
time_top = np.array([11+42/60+47/3600, 11+44/60+8/3600, 11+56/60+50/3600, 11+57/60+37/3600, 12+4/60+53/3600, 12+6/60+3/3600]) - 2 # in h, UT
z_b_top = np.array([39+9/60+32/3600, 39+3/60+16/3600, 38+6/60+6/3600, 38+3/60+7/3600, 37+35/60+57/3600, 37+32/60+2/3600]) # deg

time_bottom = np.array([11+49/60+8/3600, 11+51/60+3/3600,11+59/60+29/3600, 12+1/60+11/3600]) - 2 # in h, UT
z_b_bottom = np.array([39+12/60+26/3600, 39+4/60+52/3600, 38+29/60+1/3600, 38+21/60+33/3600]) # deg

### Almanac

In [17]:
GMST_0UT = 14 + 52/60 + 28.7584/3600 # in h
alpha_5 = 2.8197997 # in h
alpha_6 = 2.8841898 # in h
delta_5 = 16.262748 # in deg
delta_6 = 16.545533 # in deg
phi_1 = 15/60 + 51.58/3600 # in deg
phi_2 = 8.72 # in deg

l = 13 + 17/60 + 42.5/3600 # in deg

### Corrections

#### Index error

In [18]:
z_0 = np.array([85+35/60+37/3600, 85+35/60+59/3600, 85+35/60+26/3600, 85+35/60+28/3600]) # in deg
z_180 = np.array([274+23/60+44/3600, 274+23/60+53/3600, 274+23/60+24/3600, 274+22/60+41/3600]) # in deg
i = np.mean((360 - (z_0 + z_180))/2) # in deg

#### Refraction

In [19]:
# z_b is in '' but we want °
R_mean_top = 1/np.tan(((90-z_b_top/3600) + (7.31)/ (90-z_b_top/3600+4.4)) * np.pi/180)
R_mean_bottom = 1/np.tan(((90-z_b_bottom/3600) + (7.31)/ (90-z_b_bottom/3600+4.4)) * np.pi/180)

P_mean = 0.5*(101.83 + 101.80 ) # kPa
T_mean = 0.5*(11 + 13) #°C

R_top = R_mean_top *(P_mean/101 * (273+T_mean)/283)
R_bottom = R_mean_bottom *(P_mean/101 * (273+T_mean)/283)

#### Transform to center of solar disk

In [20]:
z_top = z_b_top #+ i + R_top + phi_1 - phi_2
z_bottom = z_b_bottom #+ i + R_bottom - phi_1 - phi_2

In [21]:
z = np.concatenate((z_top, z_bottom)) # in deg
time = np.concatenate((time_top, time_bottom)) # in h

### Interpolation alpha

In [22]:
alpha = alpha_5 + time/(24) * (alpha_6 - alpha_5) # in h

### Interpolation delta

In [23]:
delta = delta_5 + time/(24) * (delta_6 - delta_5) # in deg

In [24]:
# delta = delta_5/15 + time/(24) * (delta_6/15 - delta_5/15) # in h

### Hour Angle in arcsec

In [25]:
tau = GMST_0UT + time * (366.24/365.24) + l/15 - alpha # alles in h

In [26]:
Y = np.arctan(np.cos(tau*15*np.pi/180) / np.tan(delta * np.pi/180)) # rad
X = np.sin(delta*np.pi/180) * np.sqrt(1 + (np.cos(tau*15*np.pi/180)/ np.tan(delta*np.pi/180))**2)

### Prüfe welche Gleichung

In [27]:
b = (52 + 27/60 + 24/3600)*np.pi/180 # rad
print((Y + b) > np.pi/2)

[ True  True  True  True  True  True  True  True  True  True]


-> First solution for b!

In [28]:
b = (np.pi - np.arcsin(np.cos(z*np.pi/180)/X) - Y) * 180/np.pi
b

array([52.12995028, 52.13420968, 52.12144792, 52.12564703, 52.14393375,
       52.14991707, 52.75220946, 52.7670207 , 52.7417163 , 52.72408377])